### Recruit Restaurant Visitor Forecasting

In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd .read_csv)
from pandas import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default = 'warn'

In [2]:
from subprocess import check_output
print(check_output(["ls", "/Users/andoutakaaki/DIC_study/sprint8/input"]).decode("utf8"))

air_reserve.csv
air_store_info.csv
air_visit_data.csv
date_info.csv
hpg_reserve.csv
hpg_store_info.csv
sample_submission.csv
store_id_relation.csv



In [3]:
air_reserve = pd.read_csv("./input/air_reserve.csv")
air_info = pd.read_csv("./input/air_store_info.csv")
# visit_dateはdatetime型として扱えるようにparse_datesと定義
air_data = pd.read_csv("./input/air_visit_data.csv", parse_dates=['visit_date'])
date_info = pd.read_csv("./input/date_info.csv")
hpg_reserve = pd.read_csv("./input/hpg_reserve.csv")
hpg_store = pd.read_csv("./input/hpg_store_info.csv")
test_df = pd.read_csv("./input/sample_submission.csv")
store_id = pd.read_csv("./input/store_id_relation.csv")

### データセットの内容
全部で8つ。      
データは大きく分けて2種類あり、「ホットペッパー」と「Airレジ」（Airレジとは、リクルートライフスタイルが提供する、POSレジのアプリ。 スマホやタブレットなどのモバイルデバイスでPOSの導入を実現）      
各サービスの2016年〜2017年4月までの予約情報（予約日時や予約人数）が提供されている。

#### air_reserve.csv　
Airレジ経由の予約情報       
air_store_id – Airレジ固有のレストランID         
visit_datetime – 予約時のお店訪問予定時間        
reserve_datetime – 予約をした時の時間        
reserve_visitors – 予約人数

#### air_store_info.csv
Airレジのレストラン情報        
air_store_id – Airレジ固有ID    
air_genre_name – レストランのジャンル   
air_area_name – レストランの所属エリア    
latitude – 緯度    
longitude – 経度

#### air_visit_data.csv
Airレジの各レストランの日付ごとの実客数   
air_store_id – AirレジID   
visit_date – 日付   
visitors – 実客数

#### data_info.csv
カレンダー日付の基本的な情報

#### hpg_reserve.csv
ホットペッパー経由の予約情報    
hpg_store_id – ホットペッパー固有のレストランID    
visit_datetime – 予約時のお店訪問予定時間      
reserve_datetime – 予約をした時の時間     
reserve_visitors – 予約人数

#### hpg_store_info.csv
ホットペッパーのレストラン情報     
hpg_store_id – ホットペッパー固有ID     
air_genre_name – レストランのジャンル     
air_area_name – レストランの所属エリア      
latitude – 緯度     
longitude – 経度

#### sample_submission.csv
id – air_store_idとvisit_dateを連結させたid      
visitors – 予想客数（店＆日付のコンビネーション）

#### store_id_relation.csv
AirレジとホットペッパーのIDリレーション     
両サービスを使っているお店のみ     
hpg_store_id – ホットペッパー固有ID   
air_store_id – Airレジ固有ID

In [4]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [5]:
air_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [6]:
air_data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [7]:
air_data.shape

(252108, 3)

In [8]:
# レストランIDのひとつの実客数をみてみる
check_store = air_data[air_data['air_store_id'] == 'air_ba937bf13d40fb24']
check_store.describe()

,visitors
count,391.000000
mean,22.782609
std,11.810526
min,1.000000
25%,14.000000
50%,22.000000
75%,30.000000
max,61.000000


レストランID「air_ba937bf13d40fb24」では、平均22.7名、標準偏差11.81、最小1名〜最大61名が来店

In [9]:
 # visit_dateの確認
check_store.visit_date.describe()

count                     391
unique                    391
top       2016-09-08 00:00:00
freq                        1
first     2016-01-13 00:00:00
last      2017-04-22 00:00:00
Name: visit_date, dtype: object

「2016年1月13日〜2017年4月22日」の期間のデータ

In [10]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [11]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [12]:
hpg_store.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [13]:
test_df.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [14]:
# 元々のIDからstore_idとvisit_dateを切り出します
test_df['store_id'] = test_df['id'].str[:20]
test_df['visit_date'] = test_df['id'].str[21:]
 
# 提出ファイルのサンプルでvisitorsは意味がないのでdrop
test_df.drop(['visitors'], axis=1, inplace=True)
 
# 日付の型をobjectからdatetimeに変換
test_df['visit_date'] = pd.to_datetime(test_df['visit_date'])
 
# 念のためカラム情報の確認
test_df.info()
 
# ヘッダー情報もみておきましょう
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32019 entries, 0 to 32018
Data columns (total 3 columns):
id            32019 non-null object
store_id      32019 non-null object
visit_date    32019 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 750.5+ KB


,id,store_id,visit_date
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27


In [15]:
test_df.shape

(32019, 3)

In [16]:
store_id.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


### 評価指標
評価指標は、Airレジを使っている821店舗のレストランの2017年4月最終週〜2017年5月末日までの各日にちのお客さん来店数。sample_submission.csvに格納されている。全ての行のvisit（来客数）を予測する。

1.air_visit_dataに、過去の来客数データとして、日付、レストランID、実客数がある。曜日を追加して、レストランIDと曜日毎に実客数をまとめて中央値を算出        
2.各レストランの「曜日」「祝日フラグ」の項目にまとめて加重平均を算出       
3.中央値と加重平均で算出した数値の平均を最終の予測データとする

#### air_visit_data.csvを訓練データとして使うために前処理を行う        

1.visit_dateを基に曜日のデータ作成          
2.曜日とレストランIDを基にグルーピングして中央値を算出      

In [17]:
 pd.to_datetime(air_data['visit_date'])

0        2016-01-13
1        2016-01-14
2        2016-01-15
3        2016-01-16
4        2016-01-18
5        2016-01-19
6        2016-01-20
7        2016-01-21
8        2016-01-22
9        2016-01-23
10       2016-01-25
11       2016-01-26
12       2016-01-27
13       2016-01-28
14       2016-01-29
15       2016-01-30
16       2016-02-03
17       2016-02-04
18       2016-02-05
19       2016-02-06
20       2016-02-08
21       2016-02-09
22       2016-02-10
23       2016-02-11
24       2016-02-12
25       2016-02-13
26       2016-02-15
27       2016-02-16
28       2016-02-17
29       2016-02-18
            ...    
252078   2017-03-17
252079   2017-03-18
252080   2017-03-21
252081   2017-03-22
252082   2017-03-23
252083   2017-03-24
252084   2017-03-25
252085   2017-03-27
252086   2017-03-28
252087   2017-03-29
252088   2017-03-30
252089   2017-03-31
252090   2017-04-01
252091   2017-04-03
252092   2017-04-04
252093   2017-04-05
252094   2017-04-06
252095   2017-04-07
252096   2017-04-08


In [18]:
pd.to_datetime(test_df['visit_date'])

0       2017-04-23
1       2017-04-24
2       2017-04-25
3       2017-04-26
4       2017-04-27
5       2017-04-28
6       2017-04-29
7       2017-04-30
8       2017-05-01
9       2017-05-02
10      2017-05-03
11      2017-05-04
12      2017-05-05
13      2017-05-06
14      2017-05-07
15      2017-05-08
16      2017-05-09
17      2017-05-10
18      2017-05-11
19      2017-05-12
20      2017-05-13
21      2017-05-14
22      2017-05-15
23      2017-05-16
24      2017-05-17
25      2017-05-18
26      2017-05-19
27      2017-05-20
28      2017-05-21
29      2017-05-22
           ...    
31989   2017-05-02
31990   2017-05-03
31991   2017-05-04
31992   2017-05-05
31993   2017-05-06
31994   2017-05-07
31995   2017-05-08
31996   2017-05-09
31997   2017-05-10
31998   2017-05-11
31999   2017-05-12
32000   2017-05-13
32001   2017-05-14
32002   2017-05-15
32003   2017-05-16
32004   2017-05-17
32005   2017-05-18
32006   2017-05-19
32007   2017-05-20
32008   2017-05-21
32009   2017-05-22
32010   2017

#### 日付を曜日へ変換

In [19]:
# 日付を曜日へ変換(0が月曜日〜6が日曜日)
air_data['dow'] = pd.to_datetime(air_data['visit_date']).dt.dayofweek
test_df['dow'] = pd.to_datetime(test_df['visit_date']).dt.dayofweek

In [20]:
air_data.head()

,air_store_id,visit_date,visitors,dow
0,air_ba937bf13d40fb24,2016-01-13,25,2
1,air_ba937bf13d40fb24,2016-01-14,32,3
2,air_ba937bf13d40fb24,2016-01-15,29,4
3,air_ba937bf13d40fb24,2016-01-16,22,5
4,air_ba937bf13d40fb24,2016-01-18,6,0


In [21]:
test_df.head()

,id,store_id,visit_date,dow
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,6
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,1
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,2
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,3


In [22]:
# 過去データの曜日の中央値を算出
# pandasのagg関数で使うリストを作成
aggregation = {'visitors' : {'total_visitors' : 'median'}}
 
# air_dataからair_store_idとdowをグルーピングしてvisitorsの中央値（median）を算出
agg_data = air_data.groupby(['air_store_id', 'dow']).agg(aggregation).reset_index()
 
# agg_dataのカラム名をつける
agg_data.columns = ['air_store_id', 'dow', 'visitors']
agg_data['visitors']= agg_data['visitors']

agg_data.head(10)

,air_store_id,dow,visitors
0,air_00a91d42b08b08d9,0,19.0
1,air_00a91d42b08b08d9,1,24.5
2,air_00a91d42b08b08d9,2,28.0
3,air_00a91d42b08b08d9,3,30.0
4,air_00a91d42b08b08d9,4,35.5
5,air_00a91d42b08b08d9,5,11.0
6,air_00a91d42b08b08d9,6,2.0
7,air_0164b9927d20bcc3,0,6.0
8,air_0164b9927d20bcc3,1,8.0
9,air_0164b9927d20bcc3,2,8.0


In [23]:
# test_dfとagg_dataのstoreid_id、dowをすり合わせmergeさせる
merged = pd.merge(test_df, agg_data, how='left', left_on=['store_id', 'dow'], right_on=['air_store_id', 'dow'])
 
# idとvisitorsだけをfinalへ格納
final = merged[['id', 'visitors']]
 
# finalのヘッダー情報
final.head(10)

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,2.0
1,air_00a91d42b08b08d9_2017-04-24,19.0
2,air_00a91d42b08b08d9_2017-04-25,24.5
3,air_00a91d42b08b08d9_2017-04-26,28.0
4,air_00a91d42b08b08d9_2017-04-27,30.0
5,air_00a91d42b08b08d9_2017-04-28,35.5
6,air_00a91d42b08b08d9_2017-04-29,11.0
7,air_00a91d42b08b08d9_2017-04-30,2.0
8,air_00a91d42b08b08d9_2017-05-01,19.0
9,air_00a91d42b08b08d9_2017-05-02,24.5


In [24]:
# NaNを探す
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    return mis_val_table_ren_columns 

missing_values_table(final)

,Missing Values,% of Total Values
id,0,0.000000
visitors,341,1.064993


visitorsに３４１個（1.06%)のNaNがある。

In [25]:
# visitorsのNaNへ0を入れておく
final.fillna(0, inplace=True)
missing_values_table(final)

,Missing Values,% of Total Values
id,0,0.0
visitors,0,0.0


In [26]:
import glob
import re
# 全てのCSVをデータフレームに読み込む
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):pd.read_csv(fn) for fn in glob.glob('/Users/andoutakaaki/DIC_study/sprint8/input/*.csv')}
for k, v in dfs.items(): locals()[k] = v

print('data frames read:{}'.format(list(dfs.keys())))

data frames read:['air_store_info', 'date_info', 'store_id_relation', 'hpg_reserve', 'air_reserve', 'air_visit_data', 'sample_submission', 'hpg_store_info']


In [27]:
air_reserve = pd.read_csv("./input/air_reserve.csv")
air_info = pd.read_csv("./input/air_store_info.csv")
# visit_dateはdatetime型として扱えるようにparse_datesと定義
air_data = pd.read_csv("./input/air_visit_data.csv", parse_dates=['visit_date'])
date_info = pd.read_csv("./input/date_info.csv")
hpg_reserve = pd.read_csv("./input/hpg_reserve.csv")
hpg_store = pd.read_csv("./input/hpg_store_info.csv")
test_df = pd.read_csv("./input/sample_submission.csv")
store_id = pd.read_csv("./input/store_id_relation.csv")

In [28]:
for v in dfs.values():
    print(v)

             air_store_id         air_genre_name  \
0    air_0f0cdeee6c9bf3d7         Italian/French   
1    air_7cc17a324ae5c7dc         Italian/French   
2    air_fee8dcf4d619598e         Italian/French   
3    air_a17f0778617c76e2         Italian/French   
4    air_83db5aff8f50478e         Italian/French   
5    air_99c3eae84130c1cb         Italian/French   
6    air_f183a514cb8ff4fa         Italian/French   
7    air_6b9fa44a9cf504a1         Italian/French   
8    air_0919d54f0c9a24b8         Italian/French   
9    air_2c6c79d597e48096         Italian/French   
10   air_0164b9927d20bcc3         Italian/French   
11   air_681f96e6a6595f82         Italian/French   
12   air_ef920fa6f4b085f6         Italian/French   
13   air_d3e7b5952cd09ccb         Italian/French   
14   air_e64de0a6bf0739af         Italian/French   
15   air_4ce7b17062a1bf73         Italian/French   
16   air_290e7a57b390f78e         Italian/French   
17   air_4579cb0669fd411b         Italian/French   
18   air_63a

In [29]:
# data_infoの祝日フラグが1のデータ確認
date_info[date_info['holiday_flg'] == 1].head(10)

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
10,2016-01-11,Monday,1
41,2016-02-11,Thursday,1
79,2016-03-20,Sunday,1
80,2016-03-21,Monday,1
119,2016-04-29,Friday,1
123,2016-05-03,Tuesday,1
124,2016-05-04,Wednesday,1


#### 祝日を0として処理する

In [30]:
# date_infoから土日で祝日フラグが1を探してweekend_hdaysに格納
weekend_hdays = date_info.apply((lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
 
# date_infoの該当の箇所のフラグを1から0へ更新をする
date_info.loc[weekend_hdays, 'holiday_flg'] = 0

#### 古い日付には重みを低く、予測しなくはいけない2017年4月の直近のデータには重みを高くする
重みの計算..(該当の日付＋1　)÷ 全部の日付の個数 

In [31]:
date_info['weight'] = (date_info.index + 1) / len(date_info)
# 値が小さいのがより昔のデータ

In [32]:
date_info.head()

,calendar_date,day_of_week,holiday_flg,weight
0,2016-01-01,Friday,1,0.001934
1,2016-01-02,Saturday,0,0.003868
2,2016-01-03,Sunday,0,0.005803
3,2016-01-04,Monday,0,0.007737
4,2016-01-05,Tuesday,0,0.009671


In [33]:
date_info.tail()

,calendar_date,day_of_week,holiday_flg,weight
512,2017-05-27,Saturday,0,0.992263
513,2017-05-28,Sunday,0,0.994197
514,2017-05-29,Monday,0,0.996132
515,2017-05-30,Tuesday,0,0.998066
516,2017-05-31,Wednesday,0,1.000000


In [34]:
# air_visit_dataと重みを追加したdate_infoをマージさせてvisit_dataを作成
# visit_dataから不必要なcalendar_dateを落とす
visit_data = air_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
 
# visit_dataの実客数にnp.log1pの対数関数を使って処理
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)
 
# visit_dataの確認
visit_data.head(10)


,air_store_id,visit_date,visitors,day_of_week,holiday_flg,weight
0,air_ba937bf13d40fb24,2016-01-13,3.258097,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,3.496508,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,3.401197,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,3.135494,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,1.945910,NaN,NaN,NaN
5,air_ba937bf13d40fb24,2016-01-19,2.302585,NaN,NaN,NaN
6,air_ba937bf13d40fb24,2016-01-20,3.465736,NaN,NaN,NaN
7,air_ba937bf13d40fb24,2016-01-21,3.091042,NaN,NaN,NaN
8,air_ba937bf13d40fb24,2016-01-22,2.944439,NaN,NaN,NaN
9,air_ba937bf13d40fb24,2016-01-23,3.295837,NaN,NaN,NaN


In [35]:
# air_dataと重みを追加したdate_infoをマージさせてvisit_dataを作成
# visit_dataから不必要なcalendar_dateを落とす
visit_data = air_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
 
# visit_dataの実客数にnp.log1pの対数関数を使って処理
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)
 
# visit_dataの確認
visit_data.head(10)

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,weight
0,air_ba937bf13d40fb24,2016-01-13,3.258097,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,3.496508,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,3.401197,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,3.135494,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,1.945910,NaN,NaN,NaN
5,air_ba937bf13d40fb24,2016-01-19,2.302585,NaN,NaN,NaN
6,air_ba937bf13d40fb24,2016-01-20,3.465736,NaN,NaN,NaN
7,air_ba937bf13d40fb24,2016-01-21,3.091042,NaN,NaN,NaN
8,air_ba937bf13d40fb24,2016-01-22,2.944439,NaN,NaN,NaN
9,air_ba937bf13d40fb24,2016-01-23,3.295837,NaN,NaN,NaN


### コーディング
ある日のある店舗の実際の来店数が30人であったとするとき、横軸に推定人数 p、縦軸に指標値を配置した折れ線グラフを描け。KaggleのLeaderboardの1位のスコアの場合、実際の来店数との誤差は何人程度であるかを求めよ。

### EDA

与えられたデータを整理し、少なくとも以下のグラフや表を出力せよ。平均値だけではなく、最大値や中央値なども確認できると良い。

折れ線グラフ（横軸：日にち、縦軸：全てのAirREGI登録店の来店数の平均値）
折れ線グラフ（横軸：日にち、縦軸：AirREGIで予約された席数の平均値）
棒グラフ（横軸：曜日、縦軸：全てのAirREGI登録店の来店数の平均値）
棒グラフ（横軸：holiday_flgが0か1か、縦軸：全てのAirREGI登録店の来店数の平均値）
棒グラフ（横軸：土日祝かどうか、縦軸：全てのAirREGI登録店の来店数の平均値）
棒グラフ（横軸：AirREGI登録店のジャンル、縦軸：登録店数）
棒グラフ（横軸：AirREGI登録店の都道府県、縦軸：登録店数）
表（Hot Pepper GourmetとAirREGIの登録店舗数。重複が何店舗かの確認）
以下の関数を使いこなすことが重要である。

今回は複数のcsvファイルにデータが別れているため、ある列を軸に結合するためにmergeを使用する。
また、条件ごとの来店数の平均値などを求めるためには、groupbyを使用する。